In [29]:
import numpy as np
import numpy.matlib
import time
from WLM import WLChain
import matplotlib.pyplot as plt
import pyvista as pv

In [39]:
def plot_mpl(chain, filename=[], show_axes=1, save=0, end=1):
    """
    Plot polymer chain.
    
    Args:
        filename: str
            path of the generated figure
            
        show_axes: boolean
        
        save: boolean
        
        end: boolean
            whether to display the end-point of loop
    """
    
    #plt.close('all')
    fig = plt.figure(figsize=(6, 6),dpi=192)
    ax = fig.add_subplot(projection='3d')
    
    ax.plot(chain.Cc[0,:chain.N1],chain.Cc[1,:chain.N1],chain.Cc[2,:chain.N1], 
            '-', color='#D00000', linewidth=2, alpha = 0.75)
    ax.plot(chain.Cc[0,chain.N1:],chain.Cc[1,chain.N1:],chain.Cc[2,chain.N1:], 
            '-', color='#0000D0', linewidth=2, alpha = 0.75)
    # ax.plot(chain.Cc[0,:],chain.Cc[1,:],chain.Cc[2,:], 
    #         'o', markeredgecolor='#800000', markerfacecolor='#D00000')
    
    # plot chain end
    if end==1:
        ax.plot(chain.Cc[0,0],chain.Cc[1,0],chain.Cc[2,0], 
                    'o', markeredgecolor='#008000', markerfacecolor='#00D000')
        ax.plot(chain.Cc[0,-1],chain.Cc[1,-1],chain.Cc[2,-1], 
                    'o', markeredgecolor='#008000', markerfacecolor='#00D000')
    
    #CM = np.mean(Cc_backbone,axis=1)
    CT = np.array([np.max(chain.Cc[0,:])+np.min(chain.Cc[0,:]),
                    np.max(chain.Cc[1,:])+np.min(chain.Cc[1,:]),
                    np.max(chain.Cc[2,:])+np.min(chain.Cc[2,:])])/2
    d_box = np.max([np.max(chain.Cc[0,:])-np.min(chain.Cc[0,:]),
                    np.max(chain.Cc[1,:])-np.min(chain.Cc[1,:]),
                    np.max(chain.Cc[2,:])-np.min(chain.Cc[2,:])])

    if show_axes==0:
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_zticklabels([])
        ax.axis('off')

    ax.set_xlim([CT[0]-d_box/2, CT[0]+d_box/2])
    ax.set_ylim([CT[1]-d_box/2, CT[1]+d_box/2])
    ax.set_zlim([CT[2]-d_box/2, CT[2]+d_box/2])
    ax.set_box_aspect([1,1,1])

    if save==1:
        plt.savefig(filename)

    plt.show()

In [70]:
n_plot = 1
plt.close('all')
for i in range(n_plot):
    #%% test
    # backbone
    # Coordinate of C atoms in each unit
    # unit_C = load('b_c.dat')';
    unit_C = np.zeros((3,1)) # coordinate of C atoms in each unit
    
    # Degree of polymerization
    N_backbone = 5000
    
    # Chain stiffness
    a_backbone = np.array([2e2,2e1])
    
    # Unit persistence
    lambda_backbone = 1
    
    # call class
    chain01 = WLChain(N_backbone,a_backbone,lambda_backbone,unit_C)
    tStart = time.time()
    chain01.d_exc = chain01.a*0.1*2
    chain01.apply_SA = 1
    
    # # chain_grid method
    # chain01.d_exc = 1
    # chain01.kappa = 5
    # chain01.epsilon = 0.1
    # chain01.chain_grid()
    
    chain01.f = 0.2
    chain01.chain_block()
    tEnd = time.time()
    print("It cost %f sec" % (tEnd - tStart))
    # print('contour length = {:0.1f}'.format(chain01.l_contour))
    # print('persistence length = {:0.1f}'.format(chain01.l_prstnc))
    # print('end-to-end distance = {:0.1f}'.format(chain01.l_end2end))
    # print('Rg = {:0.1f}'.format(chain01.Rg))
    filename_chain = './figures/chain/2023Jun/chain_block_{:d}.png'.format(i+1)
    # chain01.plot_block(filename=filename_chain, show_axes=0, save=0, end=0)

    # plot_mpl(chain01, filename=filename_chain, show_axes=0, save=1, end=0)


    #%% Pyvista plots
    def lines_from_points(points):
        """Given an array of points, make a line set"""
        poly = pv.PolyData()
        poly.points = points
        cells = np.full((len(points) - 1, 3), 2, dtype=np.int_)
        cells[:, 1] = np.arange(0, len(points) - 1, dtype=np.int_)
        cells[:, 2] = np.arange(1, len(points), dtype=np.int_)
        poly.lines = cells
        return poly

    def polyline_from_points(points):
        poly = pv.PolyData()
        poly.points = points
        the_cell = np.arange(0, len(points), dtype=np.int_)
        the_cell = np.insert(the_cell, 0, len(points))
        poly.lines = the_cell
        return poly

    points_segment_1 = chain01.Cc[:,:chain01.N1].T
    points_segment_2 = chain01.Cc[:,chain01.N1:].T

    spline_1 = pv.Spline(points_segment_1, 2000)
    spline_2 = pv.Spline(points_segment_2, 2000)

    spline_1["radius"] = np.ones(2000)*10
    spline_2["radius"] = np.ones(2000)*5
    tube_1 = spline_1.tube(scalars="radius", absolute=True)
    tube_2 = spline_2.tube(scalars="radius", absolute=True)

    pv.set_plot_theme('document')
    pl = pv.Plotter(window_size=[800, 800])
    pl.enable_anti_aliasing('msaa')
    pl.add_mesh(tube_1, color='#FF0000',
                ambient=0.2, diffuse=0.8, specular=0.1, specular_power=10)
    pl.add_mesh(tube_2, color='#0000FF',
                ambient=0.2, diffuse=0.8, specular=0.1, specular_power=10)

    # light setting
    light = pv.Light()
    light.set_direction_angle(21, -55.0)
    light.attenuation_values = (0,0,2)
    pl.add_light(light)

    pl.show()

    







retry (1)
retry (end)
retry (1)
retry (end)
retry (1)
retry (end)
retry (1)
retry (end)
retry (1)
retry (end)
retry (1)
retry (end)
retry (1)
retry (end)
retry (1)
retry (end)
retry (1)
retry (2)
retry (3)
retry (end)
retry (1)
retry (end)
retry (1)
retry (2)
retry (3)
retry (4)
retry (5)
retry (6)
retry (7)
retry (8)
retry (9)
retry (10)
retry (11)
retry (12)
retry (13)
retry (14)
retry (15)
retry (16)
retry (17)
retry (18)
retry (19)
retry (20)
retry (21)
retry (22)
retry (end)
retry (1)
retry (end)
retry (1)
retry (end)
retry (1)
retry (end)
retry (1)
retry (end)
retry (1)
retry (end)
retry (1)
retry (end)
retry (1)
retry (end)
retry (1)
retry (2)
retry (3)
retry (end)
retry (1)
retry (end)
retry (1)
retry (end)
retry (1)
retry (end)
retry (1)
retry (end)
retry (1)
retry (end)
retry (1)
retry (end)
retry (1)
retry (end)
retry (1)
retry (end)
retry (1)
retry (2)
retry (3)
retry (4)
retry (5)
retry (6)
retry (7)
retry (8)
retry (9)
retry (10)
retry (11)
retry (12)
retry (13)
retry (14

Widget(value="<iframe src='http://localhost:60975/index.html?ui=P_0x29a5494ddc0_25&reconnect=auto' style='widt…